**Idea**: Implement Context Encoder + SRCNN  
**Framework**: Keras-Core  
**Date**: 02/11/2023  

In [6]:
'general use imports'
import os
import wget
import tarfile

'machine learning imports'
import keras_core as keras
os.environ["KERAS_BACKEND"] = "tensorflow"

### Data

In [11]:
# Downloading the data
url_voc = "http://host.robots.ox.ac.uk/pascal/VOC/voc2007/VOCtrainval_06-Nov-2007.tar"
os.makedirs(os.path.join(".", "data_voc2007"), exist_ok=True)
wget.download(url = url_voc, out = "data_voc2007")

# Extracting the data
tar = tarfile.open("data_voc2007/VOCtrainval_06-Nov-2007.tar")
tar.extractall("data_voc2007")
tar.close()
os.remove("data_voc2007/VOCtrainval_06-Nov-2007.tar")

'data_voc2007/VOCtrainval_06-Nov-2007.tar'

In [24]:
# Extracting the data
tar = tarfile.open("data_voc2007/VOCtrainval_06-Nov-2007.tar")
tar.extractall("data_voc2007")
tar.close()
os.remove("data_voc2007/VOCtrainval_06-Nov-2007.tar")

In [22]:
class UNet(keras.Model):
    """	
    U-Net model class.
    """
    def __init__(self, conv_type='Conv2D', activation='LeakyReLU', 
    residual=False, attention=False, filters=64, filter_size=3, num_classes=1):
        super(UNet, self).__init__()
        self.conv_type = conv_type
        self.activation = activation
        self.residual = residual
        self.attention = attention
        self.filters = filters
        self.filter_size = filter_size
        self.num_classes = num_classes

    def call(self, inputs, training=False):
        # Encoder
        x = self.conv_block(inputs, self.filters, self.filter_size)
        p1 = self.conv_block(x, self.filters*2, self.filter_size)
        p2 = self.conv_block(p1, self.filters*4, self.filter_size)
        p3 = self.conv_block(p2, self.filters*8, self.filter_size)

        # Bottleneck
        x = self.conv_block(p3, self.filters*16, self.filter_size)

        # Decoder
        x = self.up_conv(x, self.filters*8, self.filter_size)
        x = Concatenate(axis=-1)([x, p3])
        x = self.conv_block(x, self.filters*8, self.filter_size)

        x = self.up_conv(x, self.filters*4, self.filter_size)
        x = Concatenate(axis=-1)([x, p2])
        x = self.conv_block(x, self.filters*4, self.filter_size)

        x = self.up_conv(x, self.filters*2, self.filter_size)
        x = Concatenate(axis=-1)([x, p1])
        x = self.conv_block(x, self.filters*2, self.filter_size)

        x = self.up_conv(x, self.filters, self.filter_size)
        x = Concatenate(axis=-1)([x, inputs])
        x = self.conv_block(x, self.filters, self.filter_size)

        # Output
        activation = 'sigmoid' if self.num_classes == 1 else 'softmax'
        outputs = Conv2D(self.num_classes, (1, 1), activation=activation)(x)

        return outputs

    def conv_block(self, inputs, filters, filter_size):
        x = self.conv_layer(inputs, filters, filter_size)
        x = Activation(self.activation)(x)
        x = self.conv_layer(x, filters, filter_size)
        x = Activation(self.activation)(x)
        return x

    def conv_layer(self, inputs, filters, filter_size):
        if self.conv_type == 'Conv2D':
            return Conv2D(filters, (filter_size, filter_size), padding='same')(inputs)
        elif self.conv_type == 'DepthwiseSeparableConv2D':
            return DepthwiseSeparableConv2D(filters, (filter_size, filter_size), padding='same')(inputs)
        elif self.conv_type == 'DilatedConv2D':
            return DilatedConv2D(filters, (filter_size, filter_size), dilation_rate=2, padding='same')(inputs)

    def up_conv(self, inputs, filters, filter_size):
        x = UpSampling2D((2, 2))(inputs)
        x = Conv2D(filters, (filter_size, filter_size), padding='same')(x)
        x = Activation(self.activation)(x)
        return x

In [23]:
model = UNet()

In [ ]:
# plot some images from the dataset
import matplotlib.pyplot as plt
import numpy as np
import cv2
import random
from tqdm import tqdm
from PIL import Image
